###  Tools

Here i am using 2 tools

1. arxiv
2. wikipedia

![image.png](attachment:image.png)

In [1]:
# %pip install arxiv

In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

## arxiv wrapper and arxi tool

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")

print(arxiv.name)

arxiv


In [4]:
arxiv.invoke("attention all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

## Wikipedia wrapper and tool

In [6]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
print(wikipedia.name)

wikipedia


In [9]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")


In [ ]:
### Tavily Search tool
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

C:\Users\virub\AppData\Local\Temp\ipykernel_13688\1798663064.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


In [11]:
tavily.invoke("Provide me the recent ai news?")

[{'title': 'The latest AI news we announced in September - Google Blog',
  'url': 'https://blog.google/technology/ai/google-ai-updates-september-2025/',
  'content': '# The latest AI news we announced in September\n\nOct 08, 2025\n\nHere’s a recap of some of our biggest AI updates from September, including an AI upgrade for Chrome, a new visual search option in AI Mode and the next step in bringing helpful robots into the physical world.\n\nSuperG\n\n## General summary [...] In September, Google significantly upgraded its AI capabilities across core services like Chrome and Search. You can now use Gemini in Chrome as an AI browsing assistant and Search has an upgraded AI Mode for visual inspiration. Also, Google DeepMind introduced new robotics models, and NotebookLM gained new learning features. [...] In September, Google significantly upgraded its AI capabilities across core services like Chrome and Search. You can now use Gemini in Chrome as an AI browsing assistant and Search has a

-------------- Tools are ready ------------------

### Combine all these tools in the list

In [12]:
tools = [arxiv, wikipedia, tavily]

In [13]:
# Initialize the LLM Model

from langchain_groq import ChatGroq

llm = ChatGroq(model="qwen/qwen3-32b")

In [14]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021EDEC231A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021EE0B116A0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
llm.invoke("what is ai")

AIMessage(content="<think>\nOkay, I need to explain what AI is. Let me start by recalling what I know. AI stands for Artificial Intelligence. It's a broad field, right? So maybe I should break it down into parts. First, define AI in general terms. Then maybe talk about the different types, like narrow AI and general AI. Wait, I think there's also a distinction between strong and weak AI. Let me make sure I'm getting that right.\n\nSo, AI is the simulation of human intelligence in machines. These machines can perform tasks that typically require human intelligence, like learning, reasoning, problem-solving, perception, and language understanding. But there's a lot under that umbrella. For example, machine learning is a subset of AI where systems learn from data. Then deep learning is a part of machine learning that uses neural networks with many layers. Oh, and there's also natural language processing, computer vision, robotics, etc.\n\nI should mention the history a bit. AI has been ar

---------------- LLM and Tools are ready now how do we combine them? see below cells ---------------

In [16]:
llm_with_tools = llm.bind_tools(tools=tools)

In [17]:
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021EDEC231A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021EE0B116A0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'arxiv', 'description': 'Query arxiv papers', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'query to look up on wikipedia', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name

In [ ]:
# Execute this cell
# Here we don't get the output instead we will get to know that which tool it is calling

llm_with_tools.invoke("What is the recent news on AI")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'ry3sn6vm4', 'function': {'arguments': '{"query":"recent news on AI"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 353, 'total_tokens': 471, 'completion_time': 0.199630881, 'prompt_time': 0.014189555, 'queue_time': 0.056831685, 'total_time': 0.213820436}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f1434dc5-21ac-40d7-a06d-a5709a504d31-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent news on AI'}, 'id': 'ry3sn6vm4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 353, 'output_tokens': 118, 'total_tokens': 471})

In [21]:
llm_with_tools.invoke("What is the latest research on Quantum computing")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sdvkhtnxm', 'function': {'arguments': '{"query":"latest research on quantum computing"}', 'name': 'arxiv'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 316, 'prompt_tokens': 354, 'total_tokens': 670, 'completion_time': 0.673790383, 'prompt_time': 0.014591255, 'queue_time': 0.055855964, 'total_time': 0.688381638}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--14b166cd-6006-4fde-82eb-181efadee5b8-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'latest research on quantum computing'}, 'id': 'sdvkhtnxm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 354, 'output_tokens': 316, 'total_tokens': 670})

In [23]:
llm_with_tools.invoke("What is Machine learning?") # it might wiki or it may give thr answer by LLM

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7p14z3mcs', 'function': {'arguments': '{"query":"Machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 297, 'prompt_tokens': 351, 'total_tokens': 648, 'completion_time': 0.965266618, 'prompt_time': 0.014041343, 'queue_time': 0.051961297, 'total_time': 0.979307961}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--310cf5fa-f463-4d97-b814-d399646a1f59-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Machine learning'}, 'id': '7p14z3mcs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 351, 'output_tokens': 297, 'total_tokens': 648})

## Langgraph workflow based on ReAct Architecture

In [24]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage # Human message or AI messages
from typing import Annotated # labelling
from langgraph.graph.message import add_messages # Reducers in Langgraph